# CODE THEORY
**Integrant:** Kenny Zhu Ye, Yovany Zhu Ye
- ISIN CODE
- EAN CODE
- ISBN CODE

## SUPPORT FUNCTION

In [536]:
def letter_to_value(l):
    return ord(l.upper()) - 55 if l.isalpha() else int(l)

def generate_code(code):
    numeric_values = []
    for char in code:
        if char.isdigit():
            numeric_values.append(int(char))
        else:   
            numeric_value = letter_to_value(char)
            if numeric_value > 9:
                numeric_values.extend(int(i) for i in str(numeric_value))
            else:
                numeric_values.append(int(numeric_value))
    return numeric_values

## ISIN CODE

In [537]:
def validate_isin(isin_code):

    validation = "invalid ISIN" 
    control_digit = None
    new_isin_code = isin_code

    if len(isin_code) == 12:
        isin_code_process = isin_code[:-1] if isin_code[-1].isalpha() else isin_code
        numeric_values = generate_code(isin_code_process)
        double_numeric_values = [sum(int(digit) for digit in str(numeric_values[i] * 2)) if i % 2 == 0 and numeric_values[i] * 2 > 9
                    else numeric_values[i] * 2 if i % 2 == 0 else numeric_values[i] for i in range(len(numeric_values))]
        
        numeric_value = sum(double_numeric_values)

        if isin_code[-1].isdigit():
            control_digit = isin_code[-1]
            validation = "invalid ISIN" if numeric_value % 10 != 0 else "valid ISIN"
        else:
            control_digit = (10 - numeric_value % 10) % 10
            validation = "valid ISIN" if numeric_value % 10 == 0 else "invalid ISIN"
            new_isin_code = isin_code_process + str(control_digit)
    else:
        numeric_values = numeric_value = double_numeric_values = control_digit = new_isin_code = validation = 'invalid ISIN'

    return numeric_values, double_numeric_values, numeric_value, validation, control_digit, new_isin_code


def found_x(isin_code):
    x_values = {sum(int(x) for x in str(2*i)): i for i in range(1, 10)}

    if isin_code[2:]:
        has_x = 'X' in isin_code[2:] or 'x' in isin_code[2:]
        x = isin_code.upper().find('X', 2) 

        modified_isin_code = isin_code[:2] + isin_code[2:].replace('X', '0').replace('x', '0')
        numeric_values, double_numeric_values, numeric_value, _, _, _ = validate_isin(modified_isin_code)
        
        if x != -1:
            numeric_values[x], double_numeric_values[x] = 'X', 'X' if x % 2 == 0 else 'X'
            calculated_digit = (10 - numeric_value % 10) % 10
            digit = x_values[calculated_digit] if x % 2 == 0 else calculated_digit

            complete_isin = list(isin_code)
            complete_isin[x] = str(digit) if has_x else complete_isin.append
            complete_isin = ''.join(complete_isin)
            
        else:
            digit = None
            complete_isin = isin_code 

    return numeric_values, double_numeric_values, numeric_value, digit, complete_isin

### Vaidate ISIN code

In [538]:
isin_example = "US7960502018"
numeric_values, double_numeric_values, numeric_value, validation, control_digit, new_isin_code = validate_isin(isin_example) 

print(f"ISIN Code: {isin_example}")
print(f"Generated numeric values: {numeric_values}")
print(f"Operated numeric values: {double_numeric_values}")
print(f"Sum of operated numeric values: {numeric_value}")
print(f"Validation: {validation}")
print(f"Control digit: {control_digit}")

ISIN Code: US7960502018
Generated numeric values: [3, 0, 2, 8, 7, 9, 6, 0, 5, 0, 2, 0, 1, 8]
Operated numeric values: [6, 0, 4, 8, 5, 9, 3, 0, 1, 0, 4, 0, 2, 8]
Sum of operated numeric values: 50
Validation: valid ISIN
Control digit: 8


### Control digit

In [539]:
isin_example = "NL001158514X"
numeric_values, double_numeric_values, numeric_value, validation, control_digit, new_isin_code = validate_isin(isin_example) 

print(f"ISIN Code: {isin_example}")
print(f"Generated numeric values: {numeric_values}")
print(f"Operated numeric values: {double_numeric_values}")
print(f"Sum of operated numeric values: {numeric_value}")
print(f"Validation: {validation}")
print(f"Control digit: {control_digit}")
print(f"New ISIN Code: {new_isin_code}")

ISIN Code: NL001158514X
Generated numeric values: [2, 3, 2, 1, 0, 0, 1, 1, 5, 8, 5, 1, 4]
Operated numeric values: [4, 3, 4, 1, 0, 0, 2, 1, 1, 8, 1, 1, 8]
Sum of operated numeric values: 34
Validation: invalid ISIN
Control digit: 6
New ISIN Code: NL0011585146


### Find X

In [540]:
isin_example = "MX20273749X7"
numeric_values, double_numeric_values, numeric_value, digit, complete_isin = found_x(isin_example)

In [541]:
print(f"ISIN Code: {isin_example}")
print(f"Generated numeric values: {numeric_values}")
print(f"Operated numeric values: {double_numeric_values}")
print(f"Sum of operated numeric values: {numeric_value}")
print(f"The digit to replace X: {digit}")
print(f"New generated ISIN: {complete_isin}")

ISIN Code: MX20273749X7
Generated numeric values: [2, 2, 3, 3, 2, 0, 2, 7, 3, 7, 'X', 9, 0, 7]
Operated numeric values: [4, 2, 6, 3, 4, 0, 4, 7, 6, 7, 'X', 9, 0, 7]
Sum of operated numeric values: 67
The digit to replace X: 6
New generated ISIN: MX2027374967


## EAN-13 CODE

In [542]:
def EAN13_validate(EAN13_code):

    validation = 'invalid EAN13'
    control_digit = None
    new_ean13_code = EAN13_code

    if len(EAN13_code) == 13:
        EAN13_code_process = EAN13_code[:-1] if EAN13_code[-1].isalpha() else EAN13_code
        numeric_values = generate_code(EAN13_code_process)
        triple_values = [numeric_values[i] * 3 if i % 2 == 1 else numeric_values[i] for i in range(len(numeric_values))]

        numeric_value = sum(triple_values)

        if EAN13_code[-1].isdigit():
            control_digit = EAN13_code[-1]
            validation = "invalid EAN13" if numeric_value % 10 != 0 else "valid EAN13"
        else:
            control_digit = (10 - numeric_value % 10) % 10
            validation = "valid EAN13" if numeric_value % 10 == 0 else "invalid EAN13"
            new_ean13_code = EAN13_code_process + str(control_digit)
    else:
        numeric_values = triple_values = numeric_value = control_digit = new_ean13_code = validation = 'invalid EAN13'

    return numeric_values, triple_values, numeric_value, validation, control_digit, new_ean13_code


def found_x(EAN13_code):
    
    x = EAN13_code.upper().find('X')
    print(x)
    modified_EAN13_code = EAN13_code.upper().replace('X', '0').replace('x', '0')
    numeric_values, triple_values, numeric_value, _, _, _ = EAN13_validate(modified_EAN13_code)
    numeric_values[x], triple_values[x] = 'X', 'X' if x % 2 == 0 else 'X'

    for digit in range(10):
        test_code = EAN13_code.upper().replace('X', str(digit))
        _, _, _, validation, _, _ = EAN13_validate(test_code)
        if validation == "valid EAN13":
            x_digit = digit
    
    complete_EAN13 = EAN13_code.upper().replace('X', str(x_digit))
    
    return numeric_values, triple_values, numeric_value, x_digit, complete_EAN13

### validate EAN-13 code

In [543]:
EAN13_example = "4902778913970"
numeric_values, triple_values, numeric_value, validate_EAN13, control_digit, new_ean13_code = EAN13_validate(EAN13_example)

In [544]:
print(f"EAN13 Code: {EAN13_example}")
print(f"Generated numeric values: {numeric_values}")
print(f"Operated numeric values: {triple_values}")
print(f"Sum of operated numeric values: {numeric_value}")
print(f"Validation: {validate_EAN13}")
print(f"Control digit: {control_digit}")

EAN13 Code: 4902778913970
Generated numeric values: [4, 9, 0, 2, 7, 7, 8, 9, 1, 3, 9, 7, 0]
Operated numeric values: [4, 27, 0, 6, 7, 21, 8, 27, 1, 9, 9, 21, 0]
Sum of operated numeric values: 140
Validation: valid EAN13
Control digit: 0


### Control digit

In [545]:
EAN13_example1 = "780389328012X"
numeric_values1, triple_values, numeric_value, validate_EAN13, control_digit, new_ean13_code = EAN13_validate(EAN13_example1)   

In [546]:
print(f"EAN13 Code: {EAN13_example1}")
print(f"Generated numeric values: {numeric_values1}")
print(f"Operated numeric values: {triple_values}")  
print(f"Sum of operated numeric values: {numeric_value}")
print(f"Validation: {validate_EAN13}")
print(f"Control digit X: {control_digit}")
print(f"New generated EAN13: {new_ean13_code}")

EAN13 Code: 780389328012X
Generated numeric values: [7, 8, 0, 3, 8, 9, 3, 2, 8, 0, 1, 2]
Operated numeric values: [7, 24, 0, 9, 8, 27, 3, 6, 8, 0, 1, 6]
Sum of operated numeric values: 99
Validation: invalid EAN13
Control digit X: 1
New generated EAN13: 7803893280121


In [547]:
EAN13_example2 = "7590167890234X"
numeric_values, triple_values, numeric_value, validate_EAN13, control_digit, new_ean13_code = EAN13_validate(EAN13_example2)   

In [548]:
print(f"EAN13 Code: {EAN13_example2}")
print(f"Generated numeric values: {numeric_values}")
print(f"Operated numeric values: {triple_values}")
print(f"Sum of operated numeric values: {numeric_value}")
print(f"Validation: {validate_EAN13}")
print(f"Control digit X: {control_digit}")
print(f"New generated EAN13: {new_ean13_code}")

EAN13 Code: 7590167890234X
Generated numeric values: invalid EAN13
Operated numeric values: invalid EAN13
Sum of operated numeric values: invalid EAN13
Validation: invalid EAN13
Control digit X: invalid EAN13
New generated EAN13: invalid EAN13


### found X 

In [549]:
EAN13_example3 = '843x554161789'
numeric_values, triple_values, numeric_value, calculated_digit, complete_EAN13 = found_x(EAN13_example3)

3


In [550]:
print(f"EAN13 Code: {EAN13_example3}")  
print(f"Generated numeric values: {numeric_values}")
print(f"Operated numeric values: {triple_values}")
print(f"Sum of operated numeric values: {numeric_value}")
print(f"The digit to replace X: {calculated_digit}")
print(f"New generated EAN13: {complete_EAN13}")

EAN13 Code: 843x554161789
Generated numeric values: [8, 4, 3, 'X', 5, 5, 4, 1, 6, 1, 7, 8, 9]
Operated numeric values: [8, 12, 3, 'X', 5, 15, 4, 3, 6, 3, 7, 24, 9]
Sum of operated numeric values: 99
The digit to replace X: 7
New generated EAN13: 8437554161789


In [551]:
EAN13_example4 = '5725351455x67'
numeric_values, triple_values, numeric_value, calculated_digit, complete_EAN13 = found_x(EAN13_example4)

10


In [552]:
print(f"EAN13 Code: {EAN13_example4}")
print(f"Generated numeric values: {numeric_values}")
print(f"Operated numeric values: {triple_values}")
print(f"Sum of operated numeric values: {numeric_value}")
print(f"The digit to replace X: {calculated_digit}")
print(f"New generated EAN13: {complete_EAN13}")

EAN13 Code: 5725351455x67
Generated numeric values: [5, 7, 2, 5, 3, 5, 1, 4, 5, 5, 'X', 6, 7]
Operated numeric values: [5, 21, 2, 15, 3, 15, 1, 12, 5, 15, 'X', 18, 7]
Sum of operated numeric values: 119
The digit to replace X: 1
New generated EAN13: 5725351455167
